#### 8) We will generate simulated data and use it to perform best subset selection

a) Create a dataset of n = 100 with a predictor, X, from a random normal distribution and a noise term

b) generate a response variable, Y, such that Y = B0 + B1X + B2X^2 + B3X^3 + error, and the Beta terms are constants of your choice

c) Use best subset selection to choose the best model from the predictors X, X^2, ..., X^10.  What is the best model according to Cp, BIC, and adjusted R^2.  Show plots to provide evidence and report the coefficients of the best model obtained.

d) Repeat c, using forward stepwise selection and using backwards selection.  How do the results compare to c?

e) Now fit a lasso model to the simulated data, using the same predictors.  Use cross validation to select the optimal lambda. Create plots of the cross validation error as a function of lambda.  Report the resulting coefficient estimates and discuss the results obtained.

f) Now generate a response vector Y according to Y = B0 + B7X^7 + error and perform best subset selection and the lasso.  Discuss results

#### 9)  We will predict the number of applicatoins received using the other variables in the college data set

a) Split the data set into a training and testing set

b) Fit a linear model using least squares on the training set, and report the test error obtained

c) Fit a ridge regression model on the training set, with lambda chosen by cross-validation.  Report the test error obtained

d) Fit a lasso model on the training set, using CV to choose lambda.  Report the test error obtained and the number of non-zero coefficient estimates

e) Fit a PCR model on the training set iwth M chosen by CV.  Report the test error obtained and the value of M selected

f) e, but use PLS

g) Comment on these 5 approaches and their accuracies

#### 10) We have seen that as the number of features used in a model increases, the training error will necessarily decrease, but the test error may not.  We will now explore this in a simulated data set.

a) Generate a data set with p = 20 features, n = 1000 observations, and an associated quantitative response generated according to Y = XB + error where B has some elements exactly equalt to 0

b) Split data into a test set of n = 100 and training in the rest

c) Perform best subset selection on the training set, plot the training MSE associated with the best model of each size

d) Plot the test MSE associated with the best model of each size from c

e) For which odel size does the test set MSE take on its minimum value?  Comment on the results.  If it takes on its minimum value for a model containing only an intercept or a model containing all the features, regenerate the data in a.

f) How does the minimum model from e compare to how the data is generated?  Comment on the coefficient values